# Rees - Jones (2017) American Economic Review
## Mistaken Play in the Deferred Acceptance Algorithm: Implications for Positive Assortative Matching

DAは選好順序が真の能力なり性質なりをすべて反映していれば、strategy-proof、すなわちマッチングに参加するプレイヤーは自身の選好を偽らずにそのまま報告することが弱支配戦略となることを保証するアルゴリズムである。

しかし、現実のマッチング問題にDAを用いると、必ずしもすべてのプレイヤーが真の選好順序を報告するわけではないことが多数の事例において報告されている。

例えば、学校と生徒のマッチングを考える。

この時学校側が生徒に対して持つ選好順序はなんらかの統一テストなどの結果であることが予想できる。しかし、一回きりの統一テストの結果が生徒の真の能力を完全に反映させるわけではないことは容易に想像できる。体調不良や問題への適性などにより真の能力順序とはいくらか異なる順序をテストの結果として得られていると想定するのがむしろ自然なことである。

本論文はこのような状況下で、生徒側が真の選好順序を報告しないことにより、「より良い学校がより良い生徒とマッチするべき（PAM）」という観点で見たときに社会厚生が改善する可能性のあることを指摘している。

具体的には、能力値の低い学生群が自身の選好を偽って報告した場合に、全員が真の選好順序を報告する倍位よりも社会厚生が高くなることをシミュレーションで示している。以下では論文中で行われたシミュレーションを、シミュレーション回数を少なくして再現する。

## 具体的な設定

10校の学校に対して100人の生徒を10人ずつマッチさせることを考える。

それぞれの真のクオリティは、

$i = 1 \dots 10$として、学校$i$のクオリティが$q_i　= \frac{11 - i}{10}$で、$j = 1 \dots 100$として、生徒$j$のクオリティが$q_j = \frac{101 - j}{100}$

である。

### 学校
学校側が生徒に対して受け取るシグナルは上の真のクオリティを用いて、$s_j= q_j + \epsilon$であるとする。ただし、$\epsilon$は標準正規分布からのランダムサンプルとする。

学校はこのシグナルにしたがって生徒に対する選好順序を構成し、デザイナーに報告する。

### 生徒
生徒側の選好順序におけるmisrepresentationは以下のようにモデル化する。

misrepresentationする生徒の割合を$\left\{0,0.1, 0.2, 0.3, 0.4, 0.5 \right\}$の6パターン用意する。

misrepresentationするかどうかが生徒の性質と送還するかどうかについて、
- 上位の生徒が選好の構成に失敗する
- 下位の生徒が選好の構成に失敗する
- 生徒はランダムに選好の構成に失敗する
の3パターンを用意する。

ここで、「ある生徒がmisrepresentationを起こす」とは、当該生徒の選好がランダムに順序交換させられることを指す。

### シミュレーション
生徒側の選好構成の失敗の仕方により、18通りのシミュレーションを行う。

それぞれの場合において1000回ランダムに選好を真のものからずらしてDAによるマッチングを行う。

それぞれの場合に対して、各学校がマッチした生徒のクオリティの平均値をプロットすることで、misrepresentationがPAMに対してどのような結果をもたらすのかを考察する。

In [103]:
using StatsBase

# school preferences
function school_pref(seed, num_stu, num_sch)
    srand(seed)
    pre_sch = ones(num_stu, 2)
    pre_sch[:, 1] = collect(range(1,num_stu))
    pre_sch[:, 2] = collect(1:-1/num_stu:1/num_stu) + randn(num_stu)
    pre_sch = sortrows(pre_sch, by=x->x[2], rev = true)
    return [round.(Int64, Array(pre_sch[:, 1])) for i in range(1, num_sch)], pre_sch
end

# student preferences
# student preferences permutation
# types = 1 means higher ability students misrepresent
# types = 2 means lower ability students misrepresent
# types = 3 means atudents misrepresent at random
function permute_prefs(seed, num_stu, num_sch, percent, types)
    srand(seed)
    
    true_prefs = [collect(1:1:num_sch) for i in range(1, num_stu)]
    if types == 1
        if percent == 0
            return true_prefs
        else
            for (i, n) in enumerate(true_prefs)
                if i <= percent
                    true_prefs[i] = n[randperm(length(n))]
                end
            end
        end
                    
    elseif types == 2
        if percent == 0
            return true_prefs
        else
            for i in num_stu - percent + 1:num_stu
                n = true_prefs[i]
                true_prefs[i] = n[randperm(length(n))]
            end
        end
        
    else
        list = sample(1:num_stu, percent, replace = false)
        for i in list
            n = true_prefs[i]
            true_prefs[i] = n[randperm(length(n))]
        end
    end
    
    return true_prefs
    
end

permute_prefs (generic function with 2 methods)

## misrepresentationの分析

In [76]:
using DefferedAcceptance

In [98]:
percents = collect(0:10:50)
cor = [1, 2, 3]
num_simu = 100
num_stu = 100
num_sch = 10
cap = 10
caps = round.(Int64, ones(num_sch)*cap);

### 能力の高い学生がmisrepresentする場合

In [104]:
types = 1
    average_mat = ones(num_sch, length(percents))
    for (k, percent) in enumerate(percents)
        score_mat = ones(num_sch, num_simu)
        for i in num_simu
            school_prefs, pre = school_pref(i, num_stu, num_sch)
            student_prefs = permute_prefs(i, num_stu, num_sch, percent, types)
            result = DefferedAcceptance.ikegami(student_prefs, school_prefs, caps)

            # average quality calculation
            pre_sch = sortrows(pre, by=x->x[1])
            resp_matched = result[2]
            indptr = result[3]
            scores = ones(num_sch)
            for j in 1:num_sch
                students_list = resp_matched[indptr[j]:indptr[j+1]-1]
                scores[j] = mean(pre_sch[students_list, 2])
            end
            score_mat[:, i] = scores
        end
        average = mean(score_mat, 2)
        average_mat[:, k] = average
    end

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Bool}, ::Int64) at ./deprecated.jl:57
 [3] ikegamida_mm(::Array{Int64,2}, ::Array{Int64,2}, ::Array{Int64,1}) at /Users/susu/.julia/v0.6/DefferedAcceptance/src/DefferedAcceptance.jl:152
 [4] ikegami(::Array{Array{Int64,1},1}, ::Array{Array{Int64,1},1}, ::Array{Int64,1}) at /Users/susu/.julia/v0.6/DefferedAcceptance/src/DefferedAcceptance.jl:39
 [5] macro expansion at ./In[104]:8 [inlined]
 [6] anonymous at ./<missing>:?
 [7] include_string(::String, ::String) at ./loading.jl:515
 [8] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/susu/.julia/v0.6/IJulia/src/execute_request.jl:160
 [9] eventloop(::ZMQ.Socket) at /Users/susu/.julia/v0.6/IJulia/src/eventloop.jl:8
 [10] (::IJulia.##11#14)() at ./task.jl:335
while loading In[104], in expression starting on line 3
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Int64}, ::Tuple{Int64,Int64}) at ./deprecated.jl:57
 [

In [105]:
average_mat

10×6 Array{Float64,2}:
 1.01534   1.01263   1.00963   1.00907   1.00843   1.008   
 1.00523   1.00331   1.00193   1.00134   0.999407  0.998433
 1.00165   0.999886  0.998178  0.996363  0.995714  0.996221
 0.998037  0.997579  0.996078  0.99422   0.993735  0.993249
 0.995928  0.997179  0.995051  0.995112  0.99521   0.992791
 0.9935    0.993405  0.992952  0.992372  0.992173  0.992995
 0.991384  0.993813  0.999399  0.998268  0.997746  0.997436
 0.988038  0.986504  0.988992  0.989787  0.992599  0.994876
 0.983264  0.987011  0.987831  0.986846  0.988355  0.988075
 0.979629  0.980673  0.981954  0.988626  0.988626  0.989923

3×2 Array{Float64,2}:
 1.0  1.0
 1.0  1.0
 1.0  1.0

In [96]:
mean(a, 2)

3×1 Array{Float64,2}:
 1.0
 1.0
 1.0